In [4]:
import sys
import torch
import tntorch as tn
import numpy as np
sys.path.append('..')
import lib

In [5]:
rho, train_X, train_y = lib.read_data('../data/2020_01_29_stabilizer_dim32_0.dat')
dim = rho.shape[0]
n_qubits = np.log2(dim)

tensor_rank = 2**n_qubits
batch_size = 100
lr = 1e-3

sigma = lib.simulator.randomMixedState(rho.shape[0])
sigma_real, sigma_imag = [tn.Tensor(x, ranks_tt=tensor_rank, requires_grad=True) for x in [np.real(sigma), np.imag(sigma)]]

epoches = 10

In [3]:
def trace(tensor):
    if len(tensor.shape) == 2:
        return sum([tensor[i,i] for i in range(tensor.shape[0])])
    if len(tensor.shape) == 3:
        return sum([tensor[:, i,i] for i in range(tensor.shape[1])])

def cholesky(sigma_real, sigma_imag):
    sigma_real, sigma_imag = sigma_real.dot(sigma_real, k=1)+sigma_imag.dot(sigma_imag, k=1), sigma_real.dot(sigma_imag, k=1)-sigma_imag.dot(sigma_real, k=1)
    trace = sum([sigma_real[i,i] for i in range(rho.shape[0])])
    sigma_real, sigma_imag = [x/trace for x in [sigma_real, sigma_imag]]
    return sigma_real, sigma_imag
    
#initial_trace = sum((np.real(trace(test_X.dot(sigma)))-test_y)**2)

def loss(sigma_real, sigma_imag):
    sigma_real, sigma_imag = cholesky(sigma_real, sigma_imag)
    res = 0
    idx = np.random.choice(np.arange(train_X.shape[0]), batch_size)
    for E_m,y_m in zip(train_X[idx], train_y[idx].astype('float64')):
        E_real, E_imag = [tn.Tensor(x) for x in [np.real(E_m), np.imag(E_m)]]
        res += ((E_real.dot(sigma_real)+E_imag.dot(sigma_imag)-y_m)**2)
#     return res/(initial_trace*train_X.shape[0])
    return res
    

def eval_loss(sigma_real, sigma_imag): # any score function can be used here
    sigma_real_, sigma_imag_ = cholesky(sigma_real, sigma_imag)
    sigma = sigma_real_.torch().detach().cpu().numpy() + 1j*sigma_imag_.torch().detach().cpu().numpy()
    return -lib.fidelity(sigma, rho)

#print('Trace before: %f'%initial_trace)

lib.tn_optimize([sigma_real, sigma_imag], loss, eval_loss, tol=0, patience=300,print_freq=10,lr=lr)

Ranger optimizer loaded. 
Gradient Centralization usage = True
GC applied to both conv and fc layers
iter: 0 |  loss: 0.0693553015589714 | eval_loss: -0.032980682197801356 | total time: 0.15485048294067383


../lib/optim.py:144: UserWarning: This overload of addcmul_ is deprecated:
	addcmul_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcmul_(Tensor tensor1, Tensor tensor2, *, Number value) (Triggered internally at  /opt/conda/conda-bld/pytorch_1595629395347/work/torch/csrc/utils/python_arg_parser.cpp:766.)
  exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)


iter: 10 |  loss: 0.10407236963510513 | eval_loss: -0.03275117826339562 | total time: 1.876960277557373
iter: 20 |  loss: 0.07292797416448593 | eval_loss: -0.032307296651789136 | total time: 3.58223819732666
iter: 30 |  loss: 0.06750720739364624 | eval_loss: -0.03192692081699592 | total time: 5.304423809051514
iter: 40 |  loss: 0.06023620069026947 | eval_loss: -0.031113293758261343 | total time: 7.051975965499878
iter: 50 |  loss: 0.05828048288822174 | eval_loss: -0.03031504667002871 | total time: 8.7970712184906
iter: 60 |  loss: 0.08885950595140457 | eval_loss: -0.029413496646986996 | total time: 10.539334774017334
iter: 70 |  loss: 0.08635924756526947 | eval_loss: -0.028157906478086886 | total time: 12.276161670684814
iter: 80 |  loss: 0.05842368304729462 | eval_loss: -0.027438845535462737 | total time: 14.016284227371216
iter: 90 |  loss: 0.08159654587507248 | eval_loss: -0.02712837810887034 | total time: 15.748546361923218
iter: 100 |  loss: 0.051363151520490646 | eval_loss: -0.02

KeyboardInterrupt: 